In [1]:
import pandas as pd
import os
example_root = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))), 'examples')

## Manual Correction Interface

### Author: Yifan Gui @ Kuan Yoow Chan group

pcnaDeep offers a semi-GUI interface for manual correction, which is based on the [Napari project](https://github.com/napari/napari).

In this tutorial, we will correct the pcnaDeep output corresponding to the demo data.

__Note:__ The interface directly operates on the _tracked object table_ and the _mask_. It is suggested to copy these files before running.  
The interface only supports undo to the last saved version.

### __\#\#\# Before moving on, please check out the demo video [here]()__. 

Pictures in this tutorial are from Fiji visualisation. Please refer to the demo video in real case.

### 1 Copy the tracked object table (and the mask if necessary)

The file `testing/MCF10A_demo_tracks_refined.csv` is alreadied copied as `MCF10A_demo_tracks_refined_corrected.csv`.

### 2 Start the interface

The script `bin/napari_vis.py` takes three commandline parameters:
1. `--track`: Tracked object table.
2. `--mask`: The object mask.
3. `--image`: The composite image of PCNA and bright field (BF) channels (PCNA at the first, BF at the last)

#### 2.1 Run the following code under `bin` to start the Napari interface (for demo data).

`python vis_napari.py --image MCF10A_demo_comp.tif --mask MCF10A_demo_mask.tif --track MCF10A_demo_tracks_refined.csv`

#### 2.2 Enter the correction interface.

Type `viewer.doCorrect()` in the console to enter pcnaDeep correction interface.

To correct the track, you should use commands as below. The response will be shown on the plot instantly.

---

### 3 Correct a mitosis mis-association

- __`r -t1 <T1> -t2 <T2> -f <F>`__ Replace the ID of track 1 (T1) with the ID of track 2 (T2) from frame (F).

    - __Note__ The new track (T2) should not overlap with the old track (T1).

- __`cp -p <P> -d <D>`__ Create new mother-daughter relationship between mother (P) and daughter (D).

- __`dp -d <D>`__ Dissociate a daughter (D) from its mother. The program will search for its mother automatically.

---

In the upper-right part of `frame 15~17`, there is a mis-associated mitosis event. The `track 43` is not associated with its daughter `track 61`  (_pcnaDeep prohibits mother and daughter co-exist in one frame_), although it is associated with the other daughter `track 96`. To correct this, we do the following two steps.

1. Replace `track 43` in `frame 16` with `track 96`. That is `r -t1 43 -t2 96 -f 15`.

2. Associate mother with its two daughters. `cp -p 43 -d 96`; `cp -p 43 -d 61` (one command at a time).

![corr_m1](assets/corr_m1.png)

__Note:__ By combining `r -t1 -t2 -f` and `c -t`, tracking errors can be corrected.

### 4 Save editing

Type in __`s`__ for saving the current editing on the object table.

Every time you save, two things will be checked:
1. Object removal from the mask.
2. Object update from the mask (e.g. change the mask shape using brush). 

Related fields in the _tracked object table_ will be calculated from the pixel level of the composite image.

Type __`revert`__ to go back to the last saved version. However, you are not allowed to do other redo operations.

### 5 Correct a mitosis mis-detection

- __`c -t <T> -f <F>`__ Create new ID for track (T) from frame (F). 
---

There is another unlinked mitosis event in `frame 124~127`, where the mother `track 29` is not broken.

To break the mitosis `track 29`, we create new ID from the cytokinesis `frame 125`.

1. Create new ID for `track 29` from `frame 125`. `c -t 29 -f 125`.

&emsp;&emsp; The return message indicates that `track 101` is created. We next associate two daughters with their mother.

2. `cp -p 29 -d 101`; `cp -p 29 -d 89`.

![corr_m2](assets/corr_m2.png)

### 6 Correct cell cycle phases

- __`cls -t <T> -l <L> -f <F>`__ Correct track with ID (T) to cell cycle phase (L) from frame (F) to the next phase transition.

- __`cls -t <T> -l <L> -f <F> -e <Fe>`__ Correct from frame (F) to frame (Fe).
- __`cls -t <T> -l <L> -f <F> -s`__ Correct the classification at frame (F) _only_.

__Note:__ 

- The legal cell cycle classification is G1, G2, S, M or E. 

- The per-frame correction does not resolve temporal relationship. It is therefore recommended to resolve the corrected table again, (see [tutorial](hack_in.ipynb)).

---

There are three ways to correct cell cycle classifications for one track
1. Correct single frame
2. Correct across a frame range
3. Correct until next phase transition (default).

Here we illustrate how to correct in a specified range. For `track 21`, we correct the wrong `G2` classification to `S` from `frame 124` to `frame 129`.

That is, `cls -t 21 -l S -f 124 -e 129`.

![corr_cls](assets/corr_cls.png)

### 7 Delete tracks

- __`del -t <T>`__ Delete track with ID (T).

- __`del -t <T> -f <F>`__ Delete track with ID (T) at time frame (F).

&emsp; &emsp; __Warning:__ After deletion, objects will be removed from the table and cannot be added back.

---

Here we delete the entire `track 38` as an example. `del -t 38`

<img src="assets/corr_del.png"  alt="deletion example" width=500 align="middle"/>

### 8 Modify object masks

User brush and other tools in the __`segm`__ layer to modify object masks. The modification will be updated every time you save.

__Warning:__ Do not leave any object unconnected, which may lead to unpredictable effects.

---

### 9 Register new objects

Similarly, you can create new cell object in the __`segm`__ layer. However, you must draw with __a unique label__.

To check available labels, use __`mx -f <F>`__ to check maximum label at frame (F). You can use any label larger than this.

After drawing the object mask, register object into the table with:

- __`b -o <O> -t <T> -f <F> -l <L>`__ Register object labeled (O) at frame (F) and cell cycle classification (L) to track (T).

---

### 10 Other commands

- __`wq`__ to save and quit.

- __`q`__ to quit without saving.

- __`div -p <P> -ds <D1,D2> -f <F>`__ 
&emsp;(Experimental) In an established mitosis relationship (parent P, daughters D1, D2 comma separated), adjust track ID to the correct cytokinesis time frame (F).

---